# Discriminator modell

Ezen notebook egy áttekintő jegyzeként szolgál a GAN háló Diszkriminátorához.
A legegyszerűbb architektúrát kívánom bemutatni, a DCGAN diszkriminátorát, amely több későbbi modell alapjául szolgált. Remek kiinduló alap lehet. A modellt a keras Sequential API segítségével állítottam össze.

In [3]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

## Architektúra

A _Diszkriminátor_ modell lényegében egy bináris osztályozó, amely eldönti, hogy a bemenetül kapott kép valódi vagy hamis. A bemeneti kép konvolúciós rétegeken megy végig, majd a kimeneti rétegen sigmoid aktivációs függvény segítségével hozza meg a döntést.
A konvolúciós rétegek segítségével a modell fel tudja tárni a képeken található jellegzetességeket. A képen található mintázatokat filterek segítségével tanulja meg, amelyek megadott kernelméretekben pásztázzák végig a bemenetet. Így az egyes konvolúciósrétegek kimeneteként egy olyan reprezentáció jön létre a bemeneti képről, amelyben érvényesülnek a pixeleket körülölelő további pixelek is. Ezáltal a kimeneti aktivációs függvény előtti réteg, a szerializáció (Flattening) kevésbé fogja a kép tartalmára vonatkozó információt rontani.
Mivel a tanítóminta képeinek pixelértékeit normalizáltuk a [-1, 1] intervallumra, így a diszkriminátor a bemenetén is ilyen tulajdonságú képeket vár. (A tanítási folyamatról a GAN notebookban számolok be részletesebben)
A Diszkriminátor felépítésben hasonlít a Generátorhoz, lényegében annak tükörképeként képzelhetjük el. Viszont a dekonvolúciós rétegek helyett konvolúciós rétegeket alkalmazunk benne, hiszen jelen esetben a felbontás-csökkentés a cél, hogy ezáltal a kép tartalmára vonatkozó információkat értelmezhető és feldolgozható formában kinyerje a végső döntéshez.

Lényeges leírni részletesen, hogy hogy épül fel?

In [4]:
from tensorflow.keras.layers import Activation
def add_downsampling_unit(model, filters, kernel_size, strides, padding):
    model.add(
        Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding, activation="relu",
            kernel_initializer="he_normal"
        )
    )

def make_discriminator_model():
    model = Sequential()

    model.add(
        Conv2D(
            filters=64,
            kernel_size=4,
            strides=2,
            input_shape=(64, 64, 3),
            padding="same", activation="relu",
            kernel_initializer="he_normal"
        )
    )

    add_downsampling_unit(model, filters=128,
                          kernel_size=4, strides=2, padding="same")

    add_downsampling_unit(model, filters=256,
                          kernel_size=4, strides=2, padding="same")

    add_downsampling_unit(model, filters=512,
                          kernel_size=4, strides=2, padding="same")
    
    model.add(
        Conv2D(
            filters=1,
            kernel_size=4,
            strides=1,
            padding="valid"
        )
    )

    model.add(Flatten())
    model.add(Activation("sigmoid"))
    return model

In [5]:
discriminator = make_discriminator_model()
print(discriminator.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        3136      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       131200    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         524544    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 512)         2097664   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 1)           8193      
_________________________________________________________________
flatten (Flatten)            (None, 1)                 0         
_________________________________________________________________
activation (Activation)      (None, 1)                 0

2022-02-21 15:34:52.504612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-21 15:34:52.504667: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-21 15:34:52.504703: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Inspiron-5558): /proc/driver/nvidia/version does not exist
2022-02-21 15:34:52.505090: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
